In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os.path as op
from pprint import pprint as pp
import scipy.stats
import seaborn as sns
import sys
pth = op.abspath("../")
print pth
sys.path.insert(0, pth)
import teigen
reload(teigen)
print teigen.__path__

/Users/mjirik/projects/teigen
Using QVariant and QString version 1
['/Users/mjirik/projects/teigen/teigen']


In [3]:
run_list = [
    # "compensation method",
    # "compensation method 1 sphere",
    # "compensation method 10 spheres",
    # "compensation method 3 spheres various radius",
    # "compensation method 1 tube",
    # "compensation method 1 tube various radius",
    # "compensation method 1 tube various length",
    #"compensation method 1 tube mix",
    # "compensation method 1 cylinder",
    # "resolution tube"
    # "compensation method 5 tubes",
    "best test", 
    # "fixed resolution"
]

In [4]:
tg = teigen.tgmain.Teigen()
config = tg.get_default_config()

In [5]:
# pp(config)
config

OrderedDict([('generators',
              OrderedDict([('Voronoi tubes',
                            OrderedDict([('element_number', 30),
                                         ('uniform_radius_distribution', True),
                                         ('normal_radius_distribution', False),
                                         ('fixed_radius_distribution', False),
                                         ('radius_distribution_minimum', 2.0),
                                         ('radius_distribution_maximum', 10.0),
                                         ('radius_distribution_mean', 5.0),
                                         ('radius_distribution_standard_deviation',
                                          5.0),
                                         ('random_generator_seed', 0)])),
                           ('Gensei', OrderedDict([('n_objects', 10)])),
                           ('Continuous tubes',
                            OrderedDict([('element_number', 3

In [6]:
np.arange(0, 10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Prepare multiple configs


In [7]:
configs = []
tg = teigen.tgmain.Teigen()

In [8]:
run_label = "tube length"
if run_label in run_list:
    for length in [5, 6, 7, 8, 10, 12, 14, 16, 20, 24, 28, 32, 40, 48]:
        for rnd in [101, 102, 103]:
            config = tg.get_default_config()
            config["generator_id"] = 3
            config["output"]["note"] = run_label
            gc = config["generators"]["Unconnected tubes"]
            gc["element_number"] = 10
            gc["orientation_anisotropic"] = False
            gc["length_distribution_mean"] = length
            gc["length_distribution_standard_deviation"] = 0.0
            gc["random_generator_seed"] = rnd
            config["postprocessing"]["measurement_resolution"] = 20
            config["appearance"]["skip_volume_generation"] = True
            configs.append(config)

In [9]:
run_label = "tube length 2"
if run_label in run_list:
    for length in [5, 10, 15, 20, 25, 30, 35, 40, 45]:
        for rnd in [101, 102, 103]:
            config = tg.get_default_config()
            config["generator_id"] = 3
            config["output"]["note"] = run_label
            gc = config["generators"]["Unconnected tubes"]
            gc["element_number"] = 10
            gc["orientation_anisotropic"] = False
            gc["length_distribution_mean"] = length
            gc["length_distribution_standard_deviation"] = 0.0
            gc["random_generator_seed"] = rnd
            config["postprocessing"]["measurement_resolution"] = 20
            config["appearance"]["skip_volume_generation"] = True
            configs.append(config)

# Vepsaná, opsaná, 

In [10]:
run_label = "compensation method"
if run_label in run_list:
    for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
        for rnd in [101]:
            for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 10
                gc["orientation_anisotropic"] = False
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 20
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

### 1 sphere

In [11]:
run_label = "compensation method 1 sphere"
if run_label in run_list:
    # for resolution in [6, 7, 8, 10, 12, 16, 18, 20, 22, 24, 26, 28, 30, 34, 38, 42, 46, 50]:
    for resolution in [6, 10, 16, 22, 26, 30, 38, 46, 50]:
        for rnd in [101, 102]:
            for radius_method in [
                "inscribed", 
                #"circumscribed", 
                #"cylinder surface", 
                "cylinder volume", 
                #"average", 
                "cylinder volume + sphere compensation"
            ]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 1
                gc["orientation_anisotropic"] = False
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 0.0
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

### 10 spheres

In [12]:
run_label = "compensation method 10 spheres"
if run_label in run_list:
    for resolution in [6, 10, 17, 25, 35, 50]:
        for rnd in [101]:
            for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 10
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 0.0
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

### 3 spheres various radius

In [13]:
run_label = "compensation method 3 spheres various radius"
import itertools
if run_label in run_list:
    resolutions = [25]
    for resolution, radius, rnd, radius_method in itertools.product(
        [25],
        [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50],
        [102, 103],
        ["inscribed", "cylinder surface", "cylinder volume", "cylinder volume + sphere compensation"]
    ):
    # for resolution in [25]:
    #     for radius in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
    #         for rnd in [102, 103]:
    #             for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average"]:
        config = tg.get_default_config()
        config["generator_id"] = 3
        config["output"]["note"] = run_label + " " + radius_method
        gc = config["generators"]["Unconnected tubes"]
        gc["element_number"] = 3
        gc["uniform_radius_distribution"] = False
        gc["fixed_radius_distribution"] = True
        gc["orientation_anisotropic"] = False
        gc["radius_distribution_mean"] = radius
        gc["radius_distribution_standard_deviation"] = 0.0
        gc["radius_distribution_maximum"] = 30.0
        gc["radius_distribution_minimum"] = 3.0
        gc["length_distribution_mean"] = 0.0
        gc["length_distribution_standard_deviation"] = 0.0
        gc["random_generator_seed"] = rnd
        config["postprocessing"]["measurement_resolution"] = resolution
        config["appearance"]["skip_volume_generation"] = True
        config["measurement"]["polygon_radius_selection_method"] = radius_method
        configs.append(config)

## 1 cylinder

In [14]:
run_label = "compensation method 1 cylinder"
import itertools
if run_label in run_list:
    resolutions = [25]
    for resolution, radius, rnd, radius_method in itertools.product(
        [10, 15, 20, 25, 35, 50],
        [10, 15],
        [102, 103],
        ["inscribed", "cylinder volume", "cylinder surface" , 
         "cylinder volume + sphere compensation", "cylinder surface + sphere compensation"]
    ):
    # for resolution in [25]:
    #     for radius in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
    #         for rnd in [102, 103]:
    #             for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average"]:
        config = tg.get_default_config()
        config["generator_id"] = 3
        config["output"]["note"] = run_label + " " + radius_method
        gc = config["generators"]["Unconnected tubes"]
        gc["element_number"] = 3
        gc["uniform_radius_distribution"] = False
        gc["fixed_radius_distribution"] = True
        gc["orientation_anisotropic"] = False
        gc["radius_distribution_mean"] = radius
        gc["radius_distribution_standard_deviation"] = 0.0
        gc["radius_distribution_maximum"] = 30.0
        gc["radius_distribution_minimum"] = 3.0
        gc["length_distribution_mean"] = 20.0
        gc["length_distribution_standard_deviation"] = 0.0
        gc["random_generator_seed"] = rnd
        config["postprocessing"]["measurement_resolution"] = resolution
        config["appearance"]["skip_volume_generation"] = True
        config["measurement"]["polygon_radius_selection_method"] = radius_method
        config["measurement"]["tube_shape"] = False
        configs.append(config)

## 1 tube

In [15]:
run_label = "compensation method 1 tube"
import itertools
if run_label in run_list:
    resolutions = [25]
    for resolution, radius, rnd, radius_method in itertools.product(
        [10, 15, 20, 25, 35, 50],
        [20, 30],
        [102, 103],
        [
            #"inscribed", "cylinder volume", "cylinder surface" , 
         "cylinder volume + sphere compensation", "cylinder surface + sphere compensation"]
    ):
    # for resolution in [25]:
    #     for radius in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
    #         for rnd in [102, 103]:
    #             for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average"]:
        config = tg.get_default_config()
        config["generator_id"] = 3
        config["output"]["note"] = run_label + " " + radius_method
        gc = config["generators"]["Unconnected tubes"]
        gc["element_number"] = 1
        gc["uniform_radius_distribution"] = False
        gc["fixed_radius_distribution"] = True
        gc["orientation_anisotropic"] = False
        gc["radius_distribution_mean"] = radius
        gc["radius_distribution_standard_deviation"] = 0.0
        gc["radius_distribution_maximum"] = 30.0
        gc["radius_distribution_minimum"] = 3.0
        gc["length_distribution_mean"] = 20.0
        gc["length_distribution_standard_deviation"] = 0.0
        gc["random_generator_seed"] = rnd
        config["postprocessing"]["measurement_resolution"] = resolution
        config["appearance"]["skip_volume_generation"] = True
        config["measurement"]["polygon_radius_selection_method"] = radius_method
        config["measurement"]["tube_shape"] = True
        configs.append(config)

In [16]:
run_label = "compensation method 1 tube problem"
if run_label in run_list:
    for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
        for rnd in [101]:
            for radius_method in [
                #"cylinder surface", 
                #"cylinder volume", 
                "cylinder volume + sphere compensation"
            ]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 1
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 20.0
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

## 1 tube various radius

In [17]:
run_label = "compensation method 1 tube various radius"
if run_label in run_list:
    # for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
    for resolution in [25]:
        for radius in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
            for rnd in [101, 102]:
                for radius_method in [
                    "inscribed",
                    "cylinder surface", 
                    "cylinder volume", 
                    "cylinder volume + sphere compensation"
                ]:
                    config = tg.get_default_config()
                    config["generator_id"] = 3
                    config["output"]["note"] = run_label + " " + radius_method
                    gc = config["generators"]["Unconnected tubes"]
                    gc["element_number"] = 1
                    gc["uniform_radius_distribution"] = False
                    gc["fixed_radius_distribution"] = True
                    gc["orientation_anisotropic"] = False
                    gc["radius_distribution_mean"] = radius
                    gc["radius_distribution_standard_deviation"] = 0.0
                    gc["radius_distribution_maximum"] = 60.0
                    gc["radius_distribution_minimum"] = 3.0
                    gc["length_distribution_mean"] = 20.0
                    gc["length_distribution_standard_deviation"] = 0.0
                    gc["random_generator_seed"] = rnd
                    config["postprocessing"]["measurement_resolution"] = resolution
                    config["appearance"]["skip_volume_generation"] = True
                    config["measurement"]["polygon_radius_selection_method"] = radius_method
                    configs.append(config)

## 1 tube various length

In [18]:
run_label = "compensation method 1 tube various length"
if run_label in run_list:
    # for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
    for resolution in [25]:
        for length in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
            for rnd in [101, 102]:
                for radius_method in [
                    #"cylinder surface", "cylinder volume", 
                    #"cylinder volume + sphere compensation",
                    "best"
                ]:
                    config = tg.get_default_config()
                    config["generator_id"] = 3
                    config["output"]["note"] = run_label + " " + radius_method
                    gc = config["generators"]["Unconnected tubes"]
                    gc["element_number"] = 1
                    gc["uniform_radius_distribution"] = False
                    gc["fixed_radius_distribution"] = True
                    gc["orientation_anisotropic"] = False
                    gc["radius_distribution_mean"] = 10.0
                    gc["radius_distribution_standard_deviation"] = 0.0
                    gc["radius_distribution_maximum"] = 60.0
                    gc["radius_distribution_minimum"] = 3.0
                    gc["length_distribution_mean"] = length
                    gc["length_distribution_standard_deviation"] = 0.0
                    gc["random_generator_seed"] = rnd
                    config["postprocessing"]["measurement_resolution"] = resolution
                    config["appearance"]["skip_volume_generation"] = True
                    config["measurement"]["polygon_radius_selection_method"] = radius_method
                    configs.append(config)

In [19]:
# 1 tube mix

In [20]:
run_label = "compensation method 1 tube mix"
import itertools
if run_label in run_list:
    resolutions = [25]
    for resolution, radius, length, rnd, radius_method in itertools.product(
        [25, 35],
        [10, 15],
        [30, 40, 50],
        [102, 103, 104, 105],
        [
            #"inscribed", "cylinder volume", "cylinder surface" , 
         "cylinder volume + sphere compensation", "cylinder surface + sphere compensation", "best"]
    ):
    # for resolution in [25]:
    #     for radius in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
    #         for rnd in [102, 103]:
    #             for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average"]:
        config = tg.get_default_config()
        config["generator_id"] = 3
        config["output"]["note"] = run_label + " " + radius_method
        gc = config["generators"]["Unconnected tubes"]
        gc["element_number"] = 1
        gc["uniform_radius_distribution"] = False
        gc["fixed_radius_distribution"] = True
        gc["orientation_anisotropic"] = False
        gc["radius_distribution_mean"] = radius
        gc["radius_distribution_standard_deviation"] = 0.0
        gc["radius_distribution_maximum"] = 30.0
        gc["radius_distribution_minimum"] = 3.0
        gc["length_distribution_mean"] = length
        gc["length_distribution_standard_deviation"] = 1.0
        gc["random_generator_seed"] = rnd
        config["postprocessing"]["measurement_resolution"] = resolution
        config["appearance"]["skip_volume_generation"] = True
        config["measurement"]["polygon_radius_selection_method"] = radius_method
        config["measurement"]["tube_shape"] = True
        configs.append(config)

# Všehochuť

In [21]:
run_label = "compensation method 5 tubes"
if run_label in run_list:
    for resolution in [10, 12, 15, 18, 21, 25, 28, 30, 35, 40, 45, 50]:
        for rnd in [101]:
            for radius_method in [
                # "cylinder surface", 
                # "cylinder volume", 
                # "cylinder volume + sphere compensation", 
                # "cylinder surface + sphere compensation", 
                "best"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 5
                gc["uniform_radius_distribution"] = True
                gc["fixed_radius_distribution"] = False
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 10
                gc["radius_distribution_standard_deviation"] = 2.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 30.0
                gc["length_distribution_standard_deviation"] = 2.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

In [22]:
# run_label = "resolution shape"
# if run_label in run_list:
#     for resolution in [5, 6, 7, 8, 10, 12, 14, 16, 20, 24, 28, 32, 40, 48]:
#         for rnd in [101, 102, 103, 104, 105]:
#             config = tg.get_default_config()
#             config["generator_id"] = 3
#             config["output"]["note"] = run_label
#             gc = config["generators"]["Unconnected tubes"]
#             gc["element_number"] = 10
#             gc["orientation_anisotropic"] = False
#             gc["length_distribution_mean"] = 0.0
#             gc["length_distribution_standard_deviation"] = 0.0
#             gc["random_generator_seed"] = rnd
#             config["postprocessing"]["measurement_resolution"] = resolution
#             config["appearance"]["skip_volume_generation"] = True
#             configs.append(config)

run_label = "resolution tube"
if run_label in run_list:
    for resolution in [6, 7, 9, 11, 13, 15, 19, 23, 28, 33, 39, 50]:
        for rnd in [102, 103]:
            config = tg.get_default_config()
            config["generator_id"] = 3
            config["output"]["note"] = run_label
            gc = config["generators"]["Unconnected tubes"]
            gc["element_number"] = 10
            gc["orientation_anisotropic"] = False
            gc["length_distribution_mean"] = 40.0
            gc["length_distribution_standard_deviation"] = 2.0
            gc["random_generator_seed"] = rnd
            config["postprocessing"]["measurement_resolution"] = resolution
            config["appearance"]["skip_volume_generation"] = True
            config["measurement"]["polygon_radius_selection_method"] = "cylinder volume + sphere compensation"
            configs.append(config)


## Best

In [23]:
run_label = "best test"
if run_label in run_list:
    for resolution in [25, 30, 35]:
        for rnd in [106, 107, 108, 109, 110]:
            for radius_method in ["best"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 1
                gc["uniform_radius_distribution"] = True
                gc["fixed_radius_distribution"] = False
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 10
                gc["radius_distribution_standard_deviation"] = 2.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 30.0
                gc["length_distribution_standard_deviation"] = 10.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

In [24]:
run_label = "fixed resolution"
if run_label in run_list:
    for resolution in [25]:
        for rnd in [106, 107, 108, 109, 110, 111, 112, 115, 116, 117, 118, 119]:
            for radius_method in [
                # "inscribed", 
                # "average",
                #"cylinder volume + sphere compensation", 
                #"cylinder surface + sphere compensation", 
                "best", 
            ]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 5
                gc["uniform_radius_distribution"] = True
                gc["fixed_radius_distribution"] = False
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 10
                gc["radius_distribution_standard_deviation"] = 2.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 30.0
                gc["length_distribution_standard_deviation"] = 2.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

# Run multiple configs

In [ ]:
#import psutil

#def print_number_of_opened_files():
#    for proc in psutil.process_iter():
#        print proc.open_files()

In [ ]:

tg = teigen.tgmain.Teigen()
for i, cfg in enumerate(configs):
    # old failing
    # tg = teigen.tgmain.Teigen()
    # config = tg.get_default_config()
    tg.use_default_config()
    tg.update_config(**cfg)
    print (str(i) + " / " + str(len(configs)))  
    print(gc["random_generator_seed"])
    print(config["postprocessing"]["measurement_resolution"])
    tg.run()
    print("run finished")
    tg.step2()
    print("step2 finished")
    # del(tg)
    

0 / 15
110
35
18.2638480731
1.0
1.0
run finished
1.0
step2 finished
1 / 15
110
35
26.5573098799
1.0
1.0
run finished
1.0
step2 finished
2 / 15
110
35
24.0726571182
31.1878402999
21.1066883411
34.0416926819
28.9826956597
1.0
1.0
run finished
1.0
step2 finished
3 / 15
110
35
32.591949486
29.2592508413
28.7385573089
32.991810029
45.782945604
31.6130870875
1.0
1.0
run finished
1.0
step2 finished
4 / 15
110
35
31.3010932678
1.0
1.0
run finished
1.0
step2 finished
5 / 15
110
35
18.2638480731
1.0
1.0
run finished
1.0
step2 finished
6 / 15
110
35
26.5573098799
1.0
1.0
run finished
1.0
step2 finished
7 / 15
110
35
24.0726571182
31.1878402999
21.1066883411
34.0416926819
28.9826956597
1.0
1.0
run finished
1.0
step2 finished
8 / 15
110
35
32.591949486
29.2592508413
28.7385573089
32.991810029
45.782945604
31.6130870875
1.0
1.0
run finished
1.0
step2 finished
9 / 15
110
35
31.3010932678
1.0
1.0
run finished
1.0
step2 finished
10 / 15
110
35
18.2638480731
1.0
1.0
run finished
1.0
step2 finished
11 / 